In [31]:
server_args = []

import math
import numpy as np
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess

proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

from pydrake.all import (LeafSystem, BasicVector, DiagramBuilder, AddMultibodyPlantSceneGraph, Parser,
                         LinearQuadraticRegulator,
                         Simulator, RigidTransform, CoulombFriction, FindResourceOrThrow, DrakeVisualizer,
                         ConnectContactResultsToDrakeVisualizer,
                         RollPitchYaw, LogVectorOutput, JointIndex, namedview,
                         Value, List, ZeroOrderHold, SpatialAcceleration, RotationMatrix, AbstractValue,
                         ConstantVectorSource)


In [46]:
def UprightState():
    state = [0, np.pi, 0, 0]
    return state


def BalancingLQR(plant):
    # Design an LQR controller for stabilizing the CartPole around the upright.
    # Returns a (static) AffineSystem that implements the controller (in
    # the original CartPole coordinates).

    context = plant.CreateDefaultContext()
    plant.get_actuation_input_port().FixValue(context, [0])
    context.get_mutable_continuous_state_vector().SetFromVector(UprightState())
    Q = np.diag((10., 10., 1., 1.))
    R = [1]

    # MultibodyPlant has many (optional) input ports, so we must pass the
    # input_port_index to LQR.
    return LinearQuadraticRegulator(
        plant,
        context,
        Q,
        R,
        input_port_index=plant.get_actuation_input_port().get_index())


class myController(LeafSystem):
    def __init__(self, K):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("u", BasicVector(4))
        self.DeclareVectorOutputPort("y", BasicVector(1), self.CalcOutputY)
        self.K = K

    def CalcOutputY(self, context, output):
        statex = self.get_input_port(0).Eval(context)
        y = -np.dot(self.K, (statex - np.array([0, np.pi, 0, 0])))
        #  print(statex, y, statex-np.array([0, np.pi, 0, 0]))
        output.SetFromVector([y])


In [56]:
from pydrake.geometry import (
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    StartMeshcat,
    Role
)

from pydrake.multibody.parsing import Parser
from pydrake.systems.framework import DiagramBuilder
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph

K = np.array([-15.29051988, 220.55525994, -18.85830785, 44.42915392])
#K = np.array([-0. ,        124.96  ,      -6.13149847,  17.06574924])

meshcat = StartMeshcat()
meshcat.Delete()
meshcat.DeleteAddedControls()

builder = DiagramBuilder()
controller = builder.AddSystem(myController(K))
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

urdf_path = "./urdfExample_cart_pole.urdf"
parser = Parser(plant, scene_graph)
parser.AddModelFromFile(urdf_path)
plant.Finalize()
# controller = builder.AddSystem(BalancingLQR(plant))

builder.Connect(plant.get_state_output_port(), controller.get_input_port(0))
builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())

logger_pendulum_state = LogVectorOutput(plant.get_state_output_port(), builder)

visual = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat
                                        , MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))
diagram = builder.Build()

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(True)
simulator.set_publish_every_time_step(True)
context = simulator.get_mutable_context()
context.SetContinuousState(UprightState() + np.array([0, 0.8, 0, 0]))
simulator.Initialize()
sim_time = 5
simulator.AdvanceTo(sim_time)

INFO:drake:Meshcat listening for connections at http://localhost:7035
